# Predicting Pulmonary Embolism from Reports
This notebook will summarize early models to predict pulmonary embolism, PE, from radiology reports. For all analyses, we aim to do the following:

1. Chapman's method (rule-based PEFinder) against Stanford data, and reproduce his result with his data
2. Stanford (machine learning) method against Stanford and Chapman data
3. Compare the two methods, and for Stanford, test using impressions (as Chapman did) and full reports

This early result is just showing a Stanford method against Stanford (impression and full reports) and Chapman data, as we need to talk with Brian (Chapman) tomorrow (Friday 10/14) to make sure we can properly reproduce what he did. 

## Data Preprocessing
The [data preprocessing](https://github.com/radinformatics/pe-predictive/blob/master/classifiers/0.reportsPrep.py) for Stanford included parsing the Stanford data (a file called `final_3.csv`, note this is not acceptable for future work) into the exact format of the Chapman data. This means that:
- `disease_state_labels` that include "definitely positive" ... "definitely_negative" are mapped onto "Neg" and "Pos", the variable `disease_state`
- `uncertainty_labels` that include "definitely negative" ... "definitely positive" are mapped onto "Yes" and "No" and nan, the variable `uncertainty`

and while all batches are saved, for the actual analyses we don't use batch 1, as it (as reported by Yu) is not independent of batch 2.

## Decision Tree Ensemble Learning
For the [Stanford analyses](https://github.com/radinformatics/pe-predictive/blob/master/classifiers/1.predictStanford.py) I do the following:

1. Filter down data matrix, X, to either impressions or full reports, and y (labels) to some metric we are interested in (eg, disease_state). 
2. Use sklearn [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) to extract tokens and counts from the reports. I set the `ngram_range` to (1,2), meaning we will use one or two words as features. I set `stop_words` to `english` meaning that a pre-defined list of English stop words will be removed from the tokens (eg, the, a)
3. Split the data into 10 training and test sets (10 fold cross validation)
4. For each training and test:
    - Build an [extra-trees-classifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html) on the training, with `class_weight` balanced meaning that it takes frequency of the labels into account.
    - Test on the test set
    - Save results, including reports we got wrong, importance ranking of top 100 features, and set sizes
    
The remainder of this report will detail these results

In [20]:
import pickle
import numpy
results = pickle.load(open('chapman_stanford_results.pkl','rb'))

In [21]:
# What do we have for categories?
print(results.keys())

dict_keys(['quality', 'uncertainty', 'historicity', 'pulmonary_embolism'])


## Predicting Pulmonary Embolism

For each of these results, we will look at the classifier performance on Chapman and Stanford data, and for Stanford this includes using impressions and full reports. The labels (rkey) of the data take the following format:

      institution|column label|report type

First let's look at overall performance, represented by scores and confusion matrices. For the confusion matrices, we will just look at the first, since each has a result for 10 cross validations.

### Performance Metrics

In [23]:
for rkey,result in results['pulmonary_embolism'].items():
    print("\n\n\n%s" %rkey)
    print("---------------------------------------------")
    print("SCORES:\n")
    print(result['scores'])
    
    meanval = numpy.mean(result['scores'])
    stdval = numpy.std(result['scores'])
    
    print("\nMean:%s\nStandard Deviation:%s" %(meanval,stdval))
    print("\nCONFUSION (one example only):\n")
    print(result['confusion'][0])
    print("\nTEST/TRAIN SIZES:\n")
    print(result['N'])





stanford|disease_state|impression
---------------------------------------------
SCORES:

[0.9, 0.9855072463768116, 0.8985507246376812, 0.855072463768116, 0.927536231884058, 0.927536231884058, 0.8985507246376812, 0.9420289855072463, 0.9565217391304348, 0.8115942028985508]

Mean:0.910289855072
Standard Deviation:0.0475063974953

CONFUSION (one example only):

     Neg  Pos
Neg   61    0
Pos    7    2

TEST/TRAIN SIZES:

   train  test
0    621    70
1    622    69
2    622    69
3    622    69
4    622    69
5    622    69
6    622    69
7    622    69
8    622    69
9    622    69



stanford|disease_state|rad_report
---------------------------------------------
SCORES:

[0.9, 0.8405797101449275, 0.8405797101449275, 0.9130434782608695, 0.927536231884058, 0.927536231884058, 0.9130434782608695, 0.8985507246376812, 0.9420289855072463, 0.8405797101449275]

Mean:0.894347826087
Standard Deviation:0.0372801966798

CONFUSION (one example only):

     Neg  Pos
Neg   63    0
Pos    7    0

TES

We see that:

- the model performs better on Stanford data, overall, although we can't say if it's significant
- within Stanford data, we do a little better when just looking at the impression section of the report
- within Stanford data, we do quite a bit worse when we expand our labels to be more specific (eg, Yes/No to probably positive, etc.)


## Misclassifications

Which ones did we get wrong? I am a bit skeptical after looking over these, because many seem like errors in labeling! For all of these, we have the labels for predictions/actual saved, and you can assume if it's wrong then it predicted "Neg" when it was "Pos" and vice versa. Again, I'm only going to show the first for each set of cross validations:

In [38]:
for rkey,result in results['pulmonary_embolism'].items():
    print("\n\n\n%s" %rkey)
    print("---------------------------------------------")
    print("MISCLASSIFICATIONS:\n")
    reports = result['misclassified'][0]['report'].tolist()
    predictions = result['misclassified'][0]['prediction'].tolist()
    actual = result['misclassified'][0]['actual'].tolist()
    misclass = "\n\n".join(["PREDICTION:%s\nACTUAL:%s\n\n %s" %(predictions[x],actual[x],reports[x]) for x in range(0,len(reports))])
    print(misclass)




stanford|disease_state|impression
---------------------------------------------
MISCLASSIFICATIONS:

PREDICTION:Neg
ACTUAL:Pos

 -***-1.  No pulmonary embolism.-***-2.  Nonspecific patchy groundglass within the lingula and left lower -***-lobe, most likely representing atelectasis. No focal pulmonary -***-consolidation.-***-3.  Stable 2 mm pulmonary nodule within the left lower lobe, stable -***-compared to January 2012. No new suspicious pulmonary nodules -***-identified.-***-   -***-Preliminary findings were posted on PACS by on-call resident Dr. -***-Thakur.-***-

PREDICTION:Neg
ACTUAL:Pos

 -***-1.  Filling defects in the anterior segmental left upper lobe artery -***-and subsegmental branches in the posterior basal segment of the right -***-lower lobe are consistent with pulmonary embolism, which are new -***-since the most recent study from May 14, 2014.-***-2.  Increased small to moderate left pleural effusion. Associated -***-passive atelectasis left lung base.-***-3.  Lingu

## Importance

How important are the features? These are the top 100 for each. Notice that we have some ngrams with numbers, and likely we want to (disclude) these in the vectorization.

In [40]:
for rkey,result in results['pulmonary_embolism'].items():
    print("\n\n\n%s" %rkey)
    print("---------------------------------------------")
    print("IMPORTANT FEATURES:\n")
    print(result['importances'][0])




stanford|disease_state|impression
---------------------------------------------
IMPORTANT FEATURES:

       importances           std                  vocabulary
4802      0.049662  6.938894e-18             adductor hiatus
4235      0.027950  0.000000e+00                      assess
4759      0.027719  0.000000e+00                          59
7324      0.026315  0.000000e+00                 seen series
11576     0.019688  0.000000e+00                  organizing
1740      0.016014  3.469447e-18                  dr minskof
12014     0.015960  3.469447e-18             mucus retention
7210      0.015233  0.000000e+00             scattered areas
7331      0.009848  0.000000e+00             second possibly
4951      0.009313  1.734723e-18              2004 discussed
5177      0.009291  1.734723e-18                conglomerate
10863     0.009073  0.000000e+00                  left right
7112      0.008539  1.734723e-18             sided pressures
5147      0.007500  8.673617e-19          

## Predicting Quality

### Performance Metrics

In [32]:
for rkey,result in results['quality'].items():
    print("\n\n\n%s" %rkey)
    print("---------------------------------------------")
    print("SCORES:\n")
    print(result['scores'])
    
    meanval = numpy.mean(result['scores'])
    stdval = numpy.std(result['scores'])
    
    print("\nMean:%s\nStandard Deviation:%s" %(meanval,stdval))
    print("\nCONFUSION (one example only):\n")
    print(result['confusion'][0])
    print("\nTEST/TRAIN SIZES:\n")
    print(result['N'])





stanford|quality|impression
---------------------------------------------
SCORES:

[0.8985507246376812, 0.7971014492753623, 0.8840579710144928, 0.9130434782608695, 0.9130434782608695, 0.9130434782608695, 0.9705882352941176, 0.8529411764705882, 0.8970588235294118, 0.8823529411764706]

Mean:0.892178175618
Standard Deviation:0.0427320126783

CONFUSION (one example only):

            Diagnostic  Limited
Diagnostic          60        0
Limited              7        2

TEST/TRAIN SIZES:

   train  test
0    617    69
1    617    69
2    617    69
3    617    69
4    617    69
5    617    69
6    618    68
7    618    68
8    618    68
9    618    68



chapman|quality|impression
---------------------------------------------
SCORES:

[0.9302325581395349, 0.8953488372093024, 0.8953488372093024, 0.872093023255814, 0.9534883720930233, 0.9186046511627907, 0.9418604651162791, 0.9302325581395349, 0.9294117647058824, 0.9176470588235294]

Mean:0.918426812585
Standard Deviation:0.0232174826938

CO

We see that:

- within Stanford, we do much better on quality (from mean of 86% to 91%) when we use impressions only
- Chapman and Stanford result are much more comparable.


## Predicting Historicity

### Performance Metrics

In [33]:
for rkey,result in results['historicity'].items():
    print("\n\n\n%s" %rkey)
    print("---------------------------------------------")
    print("SCORES:\n")
    print(result['scores'])
    
    meanval = numpy.mean(result['scores'])
    stdval = numpy.std(result['scores'])
    
    print("\nMean:%s\nStandard Deviation:%s" %(meanval,stdval))
    print("\nCONFUSION (one example only):\n")
    print(result['confusion'][0])
    print("\nTEST/TRAIN SIZES:\n")
    print(result['N'])





chapman|historicity|impression
---------------------------------------------
SCORES:

[0.8620689655172413, 1.0, 0.8620689655172413, 0.9310344827586207, 0.9310344827586207, 0.8214285714285714, 0.8214285714285714, 0.9285714285714286, 0.8928571428571429, 0.8928571428571429]

Mean:0.894334975369
Standard Deviation:0.0526765470355

CONFUSION (one example only):

     New  Old
New   25    0
Old    4    0

TEST/TRAIN SIZES:

   train  test
0    256    29
1    256    29
2    256    29
3    256    29
4    256    29
5    257    28
6    257    28
7    257    28
8    257    28
9    257    28



stanford|historicity|rad_report
---------------------------------------------
SCORES:

[0.8888888888888888, 0.7777777777777778, 0.7777777777777778, 0.6666666666666666, 0.5555555555555556, 0.75, 0.875, 0.75, 0.875, 0.625]

Mean:0.754166666667
Standard Deviation:0.105965942001

CONFUSION (one example only):

       Mixed  New
Mixed      1    1
New        0    7

TEST/TRAIN SIZES:

   train  test
0     76  

We see a big problem here - the size of the data, for Stanford. Heck, even Chapman is very small. I wouldn't use this result for anything.

## Predicting Uncertainty

In [34]:
for rkey,result in results['uncertainty'].items():
    print("\n\n\n%s" %rkey)
    print("---------------------------------------------")
    print("SCORES:\n")
    print(result['scores'])
    
    meanval = numpy.mean(result['scores'])
    stdval = numpy.std(result['scores'])
    
    print("\nMean:%s\nStandard Deviation:%s" %(meanval,stdval))
    print("\nCONFUSION (one example only):\n")
    print(result['confusion'][0])
    print("\nTEST/TRAIN SIZES:\n")
    print(result['N'])





chapman|uncertainty|impression
---------------------------------------------
SCORES:

[0.8604651162790697, 0.8604651162790697, 0.8604651162790697, 0.8372093023255814, 0.8488372093023255, 0.872093023255814, 0.8953488372093024, 0.8953488372093024, 0.8488372093023255, 0.8705882352941177]

Mean:0.864965800274
Standard Deviation:0.0181111056838

CONFUSION (one example only):

     No  Yes
No   41    3
Yes   9   33

TEST/TRAIN SIZES:

   train  test
0    773    86
1    773    86
2    773    86
3    773    86
4    773    86
5    773    86
6    773    86
7    773    86
8    773    86
9    774    85



stanford|uncertainty|rad_report
---------------------------------------------
SCORES:

[0.8695652173913043, 0.9130434782608695, 0.8985507246376812, 0.8840579710144928, 0.855072463768116, 0.927536231884058, 0.9420289855072463, 0.8985507246376812, 0.9565217391304348, 0.9130434782608695]

Mean:0.905797101449
Standard Deviation:0.0298775769972

CONFUSION (one example only):

     No  Yes
No    0  

That's all, folks!